# Week 1 - GHG Emission Prediction Internship

In [1]:
# Step 1: Import Required Libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import joblib


# Check Available Sheet Names


In [4]:
import pandas as pd

excel_file = r"C:\Users\satya\Downloads\SupplyChainEmissionFactorsforUSIndustriesCommodities.xlsx"
xls = pd.ExcelFile(excel_file)

# Show all sheet names
print(xls.sheet_names)


['Cover', 'Contents', 'Data Dictionary', 'Sources', 'LCIA Factors of Other GHGs', '2016_Summary_Commodity', '2015_Summary_Commodity', '2014_Summary_Commodity', '2013_Summary_Commodity', '2012_Summary_Commodity', '2011_Summary_Commodity', '2010_Summary_Commodity', '2016_Summary_Industry', '2015_Summary_Industry', '2014_Summary_Industry', '2013_Summary_Industry', '2012_Summary_Industry', '2011_Summary_Industry', '2010_Summary_Industry', '2016_Detail_Commodity', '2015_Detail_Commodity', '2014_Detail_Commodity', '2013_Detail_Commodity', '2012_Detail_Commodity', '2011_Detail_Commodity', '2010_Detail_Commodity', '2016_Detail_Industry', '2015_Detail_Industry', '2014_Detail_Industry', '2013_Detail_Industry', '2012_Detail_Industry', '2011_Detail_Industry', '2010_Detail_Industry', 'Sheet1']


# Step 2: Load Dataset

In [5]:
import pandas as pd

# Load Excel
excel_file = r"C:\Users\satya\Downloads\SupplyChainEmissionFactorsforUSIndustriesCommodities.xlsx"
xls = pd.ExcelFile(excel_file)

# Build list of actual sheet names to load
years = range(2010, 2017)
sheet_names = [f"{year}_Summary_Industry" for year in years]

# Parse each sheet into a dict and combine
data_by_year = {year: xls.parse(sheet_name) for year, sheet_name in zip(years, sheet_names)}
df = pd.concat(data_by_year.values(), ignore_index=True)

# Preview the combined data
df.head()


,Industry Code,Industry Name,Substance,Unit,Supply Chain Emission Factors without Margins,Margins of Supply Chain Emission Factors,Supply Chain Emission Factors with Margins,Unnamed: 7,DQ ReliabilityScore of Factors without Margins,DQ TemporalCorrelation of Factors without Margins,DQ GeographicalCorrelation of Factors without Margins,DQ TechnologicalCorrelation of Factors without Margins,DQ DataCollection of Factors without Margins
0,111CA,Farms,carbon dioxide,"kg/2018 USD, purchaser price",0.525,0.057,0.582,NaN,4,3,1,4,1
1,111CA,Farms,methane,"kg/2018 USD, purchaser price",0.029,0.001,0.030,NaN,4,3,1,1,1
2,111CA,Farms,nitrous oxide,"kg/2018 USD, purchaser price",0.003,0.000,0.003,NaN,4,3,1,4,1
3,111CA,Farms,other GHGs,"kg CO2e/2018 USD, purchaser price",0.005,0.000,0.005,NaN,3,3,1,3,1
4,113FF,"Forestry, fishing, and related activities",carbon dioxide,"kg/2018 USD, purchaser price",0.232,0.024,0.256,NaN,4,3,1,4,1


In [8]:
df_2 = pd.read_excel(excel_file, sheet_name=f'{years[0]}_Detail_Industry')
df_2.head()


,Industry Code,Industry Name,Substance,Unit,Supply Chain Emission Factors without Margins,Margins of Supply Chain Emission Factors,Supply Chain Emission Factors with Margins,Unnamed: 7,DQ ReliabilityScore of Factors without Margins,DQ TemporalCorrelation of Factors without Margins,DQ GeographicalCorrelation of Factors without Margins,DQ TechnologicalCorrelation of Factors without Margins,DQ DataCollection of Factors without Margins
0,1111A0,Oilseed farming,carbon dioxide,"kg/2018 USD, purchaser price",0.414,0.073,0.487,NaN,4,3,1,4,1
1,1111A0,Oilseed farming,methane,"kg/2018 USD, purchaser price",0.001,0.001,0.002,NaN,4,3,1,1,1
2,1111A0,Oilseed farming,nitrous oxide,"kg/2018 USD, purchaser price",0.002,0.000,0.002,NaN,4,3,1,4,1
3,1111A0,Oilseed farming,other GHGs,"kg CO2e/2018 USD, purchaser price",0.002,0.000,0.002,NaN,3,3,1,3,1
4,1111B0,Grain farming,carbon dioxide,"kg/2018 USD, purchaser price",0.680,0.082,0.762,NaN,4,3,1,4,1


In [11]:
# Step 5: Loop through each year and load both Detail_Commodity and Detail_Industry
for year in years:
    try:
        # Read Detail_Commodity sheet
        df_com = pd.read_excel(excel_file, sheet_name=f'{year}_Detail_Commodity')
        # Read Detail_Industry sheet
        df_ind = pd.read_excel(excel_file, sheet_name=f'{year}_Detail_Industry')

        # Add Source column to identify which dataset
        df_com['Source'] = 'Commodity'
        df_ind['Source'] = 'Industry'

        # Add Year column
        df_com['Year'] = year
        df_ind['Year'] = year

        # Remove leading/trailing spaces from column names
        df_com.columns = df_com.columns.str.strip()
        df_ind.columns = df_ind.columns.str.strip()

        # Rename columns to unify schema
        df_com.rename(columns={
            'Commodity Code': 'Code',
            'Commodity Name': 'Name'
        }, inplace=True)

        df_ind.rename(columns={
            'Industry Code': 'Code',
            'Industry Name': 'Name'
        }, inplace=True)

        # Concatenate both for current year and add to list
        combined = pd.concat([df_com, df_ind], ignore_index=True)
        all_data.append(combined)

    except Exception as e:
        print(f"Error processing year {year}: {e}")

In [28]:
# See shape
print(final_data.shape)

# See column names
print(final_data.columns)

# See first few rows
print(final_data.head())

# Check datatypes
print(final_data.dtypes)


(22092, 15)
Index(['code', 'name', 'substance', 'unit',
       'supply_chain_emission_factors_without_margins',
       'margins_of_supply_chain_emission_factors',
       'supply_chain_emission_factors_with_margins', 'unnamed:_7',
       'dq_reliabilityscore_of_factors_without_margins',
       'dq_temporalcorrelation_of_factors_without_margins',
       'dq_geographicalcorrelation_of_factors_without_margins',
       'dq_technologicalcorrelation_of_factors_without_margins',
       'dq_datacollection_of_factors_without_margins', 'source', 'year'],
      dtype='object')
     code                                               name       substance  \
0  1111A0  Fresh soybeans, canola, flaxseeds, and other o...  carbon dioxide   
1  1111A0  Fresh soybeans, canola, flaxseeds, and other o...         methane   
2  1111A0  Fresh soybeans, canola, flaxseeds, and other o...   nitrous oxide   
3  1111A0  Fresh soybeans, canola, flaxseeds, and other o...      other GHGs   
4  1111B0          Fresh whe

In [19]:
all_data[3]

,Code,Name,Substance,Unit,Supply Chain Emission Factors without Margins,Margins of Supply Chain Emission Factors,Supply Chain Emission Factors with Margins,Unnamed: 7,DQ ReliabilityScore of Factors without Margins,DQ TemporalCorrelation of Factors without Margins,DQ GeographicalCorrelation of Factors without Margins,DQ TechnologicalCorrelation of Factors without Margins,DQ DataCollection of Factors without Margins,Source,Year
0,1111A0,"Fresh soybeans, canola, flaxseeds, and other o...",carbon dioxide,"kg/2018 USD, purchaser price",0.373,0.072,0.444,NaN,4,3,1,4,1,Commodity,2013
1,1111A0,"Fresh soybeans, canola, flaxseeds, and other o...",methane,"kg/2018 USD, purchaser price",0.001,0.001,0.002,NaN,4,3,1,1,1,Commodity,2013
2,1111A0,"Fresh soybeans, canola, flaxseeds, and other o...",nitrous oxide,"kg/2018 USD, purchaser price",0.002,0.000,0.002,NaN,4,3,1,4,1,Commodity,2013
3,1111A0,"Fresh soybeans, canola, flaxseeds, and other o...",other GHGs,"kg CO2e/2018 USD, purchaser price",0.002,0.000,0.002,NaN,3,3,1,4,1,Commodity,2013
4,1111B0,"Fresh wheat, corn, rice, and other grains",carbon dioxide,"kg/2018 USD, purchaser price",0.722,0.079,0.801,NaN,4,3,1,4,1,Commodity,2013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3151,813B00,"Civic, social, professional, and similar organ...",other GHGs,"kg CO2e/2018 USD, purchaser price",0.008,0.000,0.008,NaN,4,3,1,5,1,Industry,2013
3152,814000,Private households,carbon dioxide,"kg/2018 USD, purchaser price",0.000,0.000,0.000,NaN,4,3,1,5,1,Industry,2013
3153,814000,Private households,methane,"kg/2018 USD, purchaser price",0.000,0.000,0.000,NaN,4,3,1,5,1,Industry,2013
3154,814000,Private households,nitrous oxide,"kg/2018 USD, purchaser price",0.000,0.000,0.000,NaN,4,3,1,5,1,Industry,2013


In [21]:
len(all_data)

14

In [22]:
# Step 6: Combine all years data into single DataFrame
final_data = pd.concat(all_data, ignore_index=True)
df.head(10)


,Industry Code,Industry Name,Substance,Unit,Supply Chain Emission Factors without Margins,Margins of Supply Chain Emission Factors,Supply Chain Emission Factors with Margins,Unnamed: 7,DQ ReliabilityScore of Factors without Margins,DQ TemporalCorrelation of Factors without Margins,DQ GeographicalCorrelation of Factors without Margins,DQ TechnologicalCorrelation of Factors without Margins,DQ DataCollection of Factors without Margins
0,111CA,Farms,carbon dioxide,"kg/2018 USD, purchaser price",0.525,0.057,0.582,NaN,4,3,1,4,1
1,111CA,Farms,methane,"kg/2018 USD, purchaser price",0.029,0.001,0.030,NaN,4,3,1,1,1
2,111CA,Farms,nitrous oxide,"kg/2018 USD, purchaser price",0.003,0.000,0.003,NaN,4,3,1,4,1
3,111CA,Farms,other GHGs,"kg CO2e/2018 USD, purchaser price",0.005,0.000,0.005,NaN,3,3,1,3,1
4,113FF,"Forestry, fishing, and related activities",carbon dioxide,"kg/2018 USD, purchaser price",0.232,0.024,0.256,NaN,4,3,1,4,1
5,113FF,"Forestry, fishing, and related activities",methane,"kg/2018 USD, purchaser price",0.002,0.000,0.002,NaN,4,3,1,1,1
6,113FF,"Forestry, fishing, and related activities",nitrous oxide,"kg/2018 USD, purchaser price",0.000,0.000,0.000,NaN,4,3,1,4,1
7,113FF,"Forestry, fishing, and related activities",other GHGs,"kg CO2e/2018 USD, purchaser price",0.003,0.000,0.003,NaN,3,3,1,3,1
8,211,Oil and gas extraction,carbon dioxide,"kg/2018 USD, purchaser price",0.381,0.060,0.441,NaN,4,3,1,3,1
9,211,Oil and gas extraction,methane,"kg/2018 USD, purchaser price",0.020,0.001,0.021,NaN,4,3,1,1,1


In [23]:
len(df)

1848

# Step 3: Data Preprocessing

In [24]:
df.columns


Index(['Industry Code', 'Industry Name', 'Substance', 'Unit',
       'Supply Chain Emission Factors without Margins',
       'Margins of Supply Chain Emission Factors',
       'Supply Chain Emission Factors with Margins', 'Unnamed: 7',
       'DQ ReliabilityScore of Factors without Margins',
       'DQ TemporalCorrelation of Factors without Margins',
       'DQ GeographicalCorrelation of Factors without Margins',
       'DQ TechnologicalCorrelation of Factors without Margins',
       'DQ DataCollection of Factors without Margins'],
      dtype='object')

In [26]:
df.isnull().sum

<bound method DataFrame.sum of       Industry Code  Industry Name  Substance   Unit  \
0             False          False      False  False   
1             False          False      False  False   
2             False          False      False  False   
3             False          False      False  False   
4             False          False      False  False   
...             ...            ...        ...    ...   
1843          False          False      False  False   
1844          False          False      False  False   
1845          False          False      False  False   
1846          False          False      False  False   
1847          False          False      False  False   

      Supply Chain Emission Factors without Margins  \
0                                             False   
1                                             False   
2                                             False   
3                                             False   
4                    

In [29]:
duplicates = final_data.duplicated().sum()
print(f"Total duplicate rows: {duplicates}")

# Remove duplicates
final_data = final_data.drop_duplicates()



Total duplicate rows: 0


In [30]:
# Clean column names
final_data.columns = final_data.columns.str.lower().str.replace(' ', '_')


In [32]:
final_data.to_csv("cleaned_emissions_data.csv", index=False)
